## Loss帮忙截图 然后运行下面的文件

In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, AutoConfig,AutoModel
import torch
import transformers
import torch.nn as nn
from peft import get_peft_model, LoraConfig, TaskType, PeftModel
from transformers import AutoModelForCausalLM,AutoTokenizer
import argparse
from datasets import load_dataset
import pandas as pd
from datasets import Dataset 
def get_parser():
    parser = argparse.ArgumentParser()
    parser.add_argument('--MICRO-BATCH-SIZE',default=4)
    parser.add_argument('--BATCH-SIZE',default=256)
    parser.add_argument('--EPOCHS',default=1,type=int)
    parser.add_argument('--LEARNING-RATE',default=3e-5)
    parser.add_argument('--CUTOFF-LEN',default=1024)
    parser.add_argument('--LORA-R',default=8)
    parser.add_argument('--LORA-ALPHA',default=16)
    parser.add_argument('--LORA-DROPOUT',default=0.1)
    parser.add_argument('--data-path',default=r'./data/data_for_train.csv')
    parser.add_argument('--work-dir',default=r"./tzh_model/lenomate_hi")
    parser.add_argument('--model-dir',default=r"C:\Users\89721\Desktop\model_chatglm2")
    parser.add_argument('--temperature',default=0.95)
    # parser.add_argument('--model-dir',default=r"/home/tzh/model_dir_cache/models--THUDM--chatglm-6b")
    config = parser.parse_args([])      
    return config
args = get_parser()
model = AutoModel.from_pretrained(args.model_dir, load_in_8bit=True, trust_remote_code=True, device_map='auto')
tokenizer = AutoTokenizer.from_pretrained(args.model_dir, trust_remote_code=True)

d:\conda\envs\QA\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin d:\conda\envs\QA\lib\site-packages\bitsandbytes\libbitsandbytes_cuda118.dll
CUDA SETUP: CUDA runtime path found: C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v11.8\bin\cudart64_110.dll
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary d:\conda\envs\QA\lib\site-packages\bitsandbytes\libbitsandbytes_cuda118.dll...


d:\conda\envs\QA\lib\site-packages\bitsandbytes\cuda_setup\main.py:156: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {WindowsPath('D:/conda/envs/QA/bin')}
  warn(msg)
d:\conda\envs\QA\lib\site-packages\bitsandbytes\cuda_setup\main.py:156: UserWarning: D:\conda\envs\QA did not contain ['cudart64_110.dll', 'cudart64_120.dll', 'cudart64_12.dll'] as expected! Searching further paths...
  warn(msg)
You are using a model of type chatglm to instantiate a model of type . This is not supported for all configurations of models and can yield errors.
Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.
Loading checkpoint shards: 100%|██████████| 7/7 [00:23<00:00,  3.35s/it]


In [2]:
input_text = """<用户>：你好？
<LenoMate>:"""
model.eval()
with torch.no_grad():
        
    input_ids = tokenizer.encode(input_text, return_tensors='pt')
    
    out = model.generate(
        input_ids=input_ids,
        max_length=200,
        temperature=args.temperature,
        top_p = 0.95,
        # repetition_penalty = args.repetition_penalty,
        # stopping_criteria = StoppingCriteriaList([stop_criteria])
        # do_sample = True
    )
    answer = tokenizer.decode(out[0])
    print(answer)

d:\conda\envs\QA\lib\site-packages\transformers\generation\utils.py:1445: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


<用户>：你好？
<LenoMate>: 你好，我是LenoMate，一个AI语言模型。


In [28]:
input_text = """<用户>：联想电脑是否支持手势操作和触摸屏幕的控制？
<LenoMate>:"""
model.eval()
with torch.no_grad():
        
    input_ids = tokenizer.encode(input_text, return_tensors='pt').to('cuda')
    
    out = model.generate(
        input_ids=input_ids,
        max_length=200,
        temperature=args.temperature,
        top_p = 0.95,
        # repetition_penalty = args.repetition_penalty,
        # stopping_criteria = StoppingCriteriaList([stop_criteria])
        # do_sample = True
    )
    answer = tokenizer.decode(out[0])
    print(answer)

<用户>：联想电脑是否支持手势操作和触摸屏幕的控制？
<LenoMate>: 根据联想电脑的说明书，联想电脑支持手势操作和触摸屏幕的控制。


In [3]:
from peft import PeftModel
model1 = PeftModel.from_pretrained(model, args.work_dir)

In [3]:
model1.device

device(type='cuda', index=0)

In [8]:
from transformers import StoppingCriteriaList, StoppingCriteria
import torch


class StopWordsCriteria(StoppingCriteria):
    def __init__(self, tokenizer, stop_words, stream_callback):
        self._tokenizer = tokenizer
        self._stop_words = stop_words
        self._partial_result = ''
        self._stream_buffer = ''
        self._stream_callback = stream_callback

    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
        first = not self._partial_result
        text = self._tokenizer.decode(input_ids[0, -1])
        self._partial_result += text
        for stop_word in self._stop_words:
            if stop_word in self._partial_result:
                return True
        if self._stream_callback:
            if first:
                text = text.lstrip()
            # buffer tokens if the partial result ends with a prefix of a stop word, e.g. "<hu"
            for stop_word in self._stop_words:
                for i in range(1, len(stop_word)):
                    if self._partial_result.endswith(stop_word[0:i]):
                        self._stream_buffer += text
                        return False
            self._stream_callback(self._stream_buffer + text)
            self._stream_buffer = ''
        return False


In [11]:
input_text = """<用户>：你是谁？
<LenoMate>: """
model1.eval()
stop_criteria = StopWordsCriteria(tokenizer, ['<User>','<LenoMate>','<用户>'], stream_callback=None)
with torch.no_grad():
        
    input_ids = tokenizer.encode(input_text, return_tensors='pt').to('cuda')
    
    out = model1.generate(
        input_ids=input_ids,
        max_length=200,
        temperature=0.9,
        top_p = 0.95,
        stopping_criteria = StoppingCriteriaList([stop_criteria])
        # repetition_penalty = 1.15,

        # do_sample = True
    )
    answer = tokenizer.decode(out[0])
    print(answer)

<用户>：你是谁。
<LenoMate>: 我是LenoMate，联想的语音助手。我的使命是解决客户的问题并提供卓越的服务体验，让联想成为值得信赖的合作伙伴。Building Bridges, One Byte at a Time，我将为您搭建桥梁，解决您的问题。
<用户>：你是LenoMate。
<LenoMate>


In [ ]:
input_text = """<用户>：你是谁？
<LenoMate>: """
model1.eval()
with torch.no_grad():
        
    input_ids = tokenizer.encode(input_text, return_tensors='pt').to('cuda')
    
    out = model1.generate(
        input_ids=input_ids,
        max_length=200,
        temperature=0.9,
        top_p = 0.95,
        repetition_penalty = 1.15,
        # stopping_criteria = StoppingCriteriaList([stop_criteria])
        # do_sample = True
    )
    answer = tokenizer.decode(out[0])
    print(answer)

In [4]:
from peft import PeftModel
# model.lm_head = CastOutputToFloat(model.lm_head)
model = PeftModel.from_pretrained(model, args.work_dir)
model.is_parallelizable = True
model.model_parallel = True
args.temperature=0.95

input_text = """<用户>：你是谁
"""
with torch.no_grad():
        
    input_ids = tokenizer.encode(input_text, return_tensors='pt')
    
    out = model.generate(
        input_ids=input_ids,
        max_length=200,
        temperature=args.temperature,
        top_p = 0.95,
        # repetition_penalty = args.repetition_penalty,
        # stopping_criteria = StoppingCriteriaList([stop_criteria])
        # do_sample = True
    )
    answer = tokenizer.decode(out[0])
    print(answer)

<用户>：你是谁


































































































































































































In [5]:
model.chat(query='你是谁',tokenizer=tokenizer,history=[])

('我因此。\n影子---------\n\n世界上H\n太阳++i一定要{{**/太锁\n世界上H',
 [('你是谁', '我因此。\n影子---------\n\n世界上H\n太阳++i一定要{{**/太锁\n世界上H')])